In [ ]:
# !pip install -qU docarray
# !pip install -qU langchain
# !pip install -qU pypdf

In [1]:
import os

from langchain.chains import RetrievalQA

from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
# from langchain.schema.document import Document
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFDirectoryLoader

In [2]:
#nome do modelo LLM
llm_model_name = os.environ.get("MODEL", "gemma")

#nome do modelo de embedding
embeddings_model_name = os.environ.get("EMBEDDINGS_MODEL_NAME",
                                       "nomic-embed-text")

 #numero de chunks que o modelo vai retornar na consulta ao vectordb
target_source_chunks = int(os.environ.get('TARGET_SOURCE_CHUNKS',4))

#url do modelo LLM, que pode ser local ou em um servidor
base_url='http://localhost:11434'

#tamanho dos chunks, nesse caso até 400 caracteres
chunk_size = 400 

#overlap entre os chunks, tentar reduzir a perda de contexto entre os chunks
chunk_overlap = 150

#diretório onde estão os arquivos para serem importados
source_directory = 'documento/' 

In [3]:
file_list = []
files = os.listdir(source_directory)

for arquivo in files:
    caminho_completo = os.path.join(source_directory, arquivo)
    if os.path.isfile(caminho_completo):
        file_list.append(caminho_completo)

print(file_list)


['documento/manual_lanternaBSU30.pdf', 'documento/manual_microondas_magico.pdf']


In [4]:
loader = PyPDFDirectoryLoader(source_directory)
docs = loader.load()
print('Total de páginas importadas:', len(docs))

Total de páginas importadas: 9


In [5]:
docs[3]

Document(page_content='\uf0b7 Com a lanterna ligada, pressione rapidamente o botão liga/desliga para alternar \nentre as funções: luz, luz piscante, sirene. \n\uf0b7 Cada pressão rápida do botão avança para a próxima função. \n \n \n3.3 Ajustar o Foco  \n\uf0b7 Para ajustar o foco do feixe de luz, gire a cabeça da lanterna para frente (zoom) \nou para trás (deszoom). \n3.4 Ativar o Laser Verde  \n\uf0b7 Para ativar o laser verde, pressione e segure o botão lateral por 2 segundos. \n\uf0b7 Solte o botão para desativar o laser verde. \n4. Segurança  \n\uf0b7 Não direcione o laser verde diretamente para os olhos.  \n\uf0b7 Mantenha a lanterna fora do alcance de crianças.  \n\uf0b7 Não utilize a lanterna em ambientes inflamáveis.  \n\uf0b7 Remova as pilhas da lanterna se não for usá-la por um longo período.  \n5. Manutenção e Cuidados  \n\uf0b7 Limpe a lente da lanterna com um pano macio e seco regularmente. \n\uf0b7 Não use produtos químicos ou abrasivos para limpar a lanterna. \n\uf0b7 A

In [6]:
embeddings = OllamaEmbeddings(base_url=base_url,
                              model=embeddings_model_name)

In [ ]:
#exemplo de como os dados são armazenados no vector db
result = embeddings.embed_query('Olá! Tudo bem com você?')
result[:5]

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                               chunk_overlap=chunk_overlap)
texts = text_splitter.split_documents(docs)
print('Total de chunks:', len(texts))

Total de chunks: 29


In [ ]:
texts[:3]

In [8]:
db = DocArrayInMemorySearch.from_documents(texts,
                                           embeddings
                                           )

d:\DADOS\Linkedin\RAG\.venv\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [11]:
llm = Ollama(base_url=base_url,
                 model=llm_model_name,
                 temperature = 0.2,
                 )

In [12]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=db.as_retriever(search_kwargs={"k": target_source_chunks}),
                                 return_source_documents=True,verbose=False,
                                 )

In [9]:
query = "Minha lanterna não liga o que pode ser?"

In [ ]:
#fazer a pesquisa no vector db com base na query
found_docs = db.similarity_search_with_score(query, k=target_source_chunks)
for document, score in found_docs:
    print(document.page_content)
    print(f"\nScore: {score}")
    print("Arquivo:",document.metadata["source"])
    print("Página:",document.metadata["page"],"\n\n")

In [14]:
res = qa.invoke(query)
answer, docs = res['result'], res['source_documents']


print("\n\n> Question:")
print(query)
print(answer)
print("\n\n>")
# exibe o trecho, nome do arquivo e a página do texto usado para a auxiliar na resposta
for document in docs:
    print("\n> " + document.metadata["source"] + ":")
    print( document.page_content)
    




> Question:
Minha lanterna não liga o que pode ser?
A lanterna não liga devido a problemas com as pilhas. Verifique se as pilhas estão instaladas corretamente ou se precisam ser substituídas por novas.


>

> documento\manual_lanternaBSU30.pdf:
 Com a lanterna ligada, pressione rapidamente o botão liga/desliga para alternar 
entre as funções: luz, luz piscante, sirene. 
 Cada pressão rápida do botão avança para a próxima função. 
 
 
3.3 Ajustar o Foco  
 Para ajustar o foco do feixe de luz, gire a cabeça da lanterna para frente (zoom) 
ou para trás (deszoom). 
3.4 Ativar o Laser Verde

> documento\manual_lanternaBSU30.pdf:
5. Manutenção e Cuidados  
 Limpe a lente da lanterna com um pano macio e seco regularmente. 
 Não use produtos químicos ou abrasivos para limpar a lanterna. 
 Armazene a lanterna em local seco e fresco quando não estiver em uso. 
6. Solução de Problemas  
Problema:  A lanterna não liga. 
Solução: 
 Verifique se as pilhas estão instaladas corretamente. 
 S